In [1]:
%matplotlib inline

import os.path as osp
import cv2
# from nuscenes import NuScenes
from nuimages import NuImages
from nuscenes.utils.geometry_utils import BoxVisibility

import matplotlib.pyplot as plt

# nusc = NuScenes(version='v1.0-trainval', dataroot='/data/datasets/nuscenes', verbose=True)
nuim = NuImages(version='v1.0-train', dataroot='/data/datasets/nuimages', verbose=True, lazy=False)


Loading nuImages tables for version v1.0-train...
Loaded 12 attribute(s) in 0.521s,
Loaded 2022 calibrated_sensor(s) in 0.071s,
Loaded 25 category(s) in 0.026s,
Loaded 872181 ego_pose(s) in 6.673s,
Loaded 350 log(s) in 0.022s,
Loaded 557715 object_ann(s) in 5.220s,
Loaded 67279 sample(s) in 0.166s,
Loaded 872181 sample_data(s) in 5.364s,
Loaded 6 sensor(s) in 0.011s,
Loaded 73755 surface_ann(s) in 4.643s,
Done loading in 22.719 seconds (lazy=False).


In [2]:
for idx_log, log in enumerate(nuim.log):
    print(idx_log, log)

0 {'token': 'ddefc6e23b5147a4bb04f51ebcb8ddb1', 'logfile': 'n003-2018-01-03-12-03-23+0800', 'location': 'singapore-onenorth', 'vehicle': 'n003', 'date_captured': '2018-01-03'}
1 {'token': '8322bb4604ad409886bb655470941a22', 'logfile': 'n003-2018-01-03-14-16-47+0800', 'location': 'singapore-onenorth', 'vehicle': 'n003', 'date_captured': '2018-01-03'}
2 {'token': 'd0e0f3306cf6413e82fc4b9ea1f1f00a', 'logfile': 'n003-2018-01-03-14-33-35+0800', 'location': 'singapore-onenorth', 'vehicle': 'n003', 'date_captured': '2018-01-03'}
3 {'token': '5928fde4e2254e4aa47e50424b8cfa6f', 'logfile': 'n003-2018-01-03-14-49-59+0800', 'location': 'singapore-onenorth', 'vehicle': 'n003', 'date_captured': '2018-01-03'}
4 {'token': '59933f0443fb401c85292dc94de5d32e', 'logfile': 'n003-2018-01-03-15-21-35+0800', 'location': 'singapore-onenorth', 'vehicle': 'n003', 'date_captured': '2018-01-03'}
5 {'token': '59dbff6c10e1490fbef4deb46613e813', 'logfile': 'n003-2018-01-03-17-00-35+0800', 'location': 'singapore-oneno

In [3]:
from typing import Tuple
import numpy as np
from nuscenes.utils.geometry_utils import view_points
from nuscenes.utils.data_classes import Box

def cv2_put_multi_line_text(im: np.ndarray,
                            text: str,
                            center: Tuple,
                            color: Tuple,
                            font=cv2.FONT_HERSHEY_COMPLEX,
                            font_scale=0.5,
                            thickness=1) -> None:
    text_size, _ = cv2.getTextSize("dummy_text", font, font_scale, thickness)
    line_height = text_size[1] + 5
    y0 = int(center[1])
    for i, text_line in enumerate(text.split('\n')):
        y = y0 + i * line_height
        cv2.putText(im, text_line, (int(center[0]), y),
                    cv2.FONT_HERSHEY_COMPLEX, font_scale, color[::-1], thickness, cv2.LINE_AA)


break_all = False


In [4]:
# font = cv2.FONT_HERSHEY_COMPLEX
# font_scale = 0.5
# thickness = 1

# def box_render_cv2_text(box: Box,
#                     nusc: NuScenes,
#                     im: np.ndarray,
#                     view: np.ndarray = np.eye(3),
#                     normalize: bool = False,
#                     colors: Tuple = ((0, 0, 255), (255, 0, 0), (155, 155, 155)),
#                     linewidth: int = 2,
#                     ) -> None:
#     """
#     Renders box using OpenCV2.
#     :param Box: Box to be rendered
#     :param nusc: Active NuScenes object
#     :param im: <np.array: width, height, 3>. Image array. Channels are in BGR order.
#     :param view: <np.array: 3, 3>. Define a projection if needed (e.g. for drawing projection in an image).
#     :param is_key_frame
#     :param normalize: Whether to normalize the remaining coordinate.
#     :param colors: ((R, G, B), (R, G, B), (R, G, B)). Colors for front, side & rear.
#     :param linewidth: Linewidth for plot.
#     """
#     corners = view_points(box.corners(), view, normalize=normalize)[:2, :]

#     def draw_rect(selected_corners, color):
#         prev = selected_corners[-1]
#         for corner in selected_corners:
#             cv2.line(im,
#                      (int(prev[0]), int(prev[1])),
#                      (int(corner[0]), int(corner[1])),
#                      color, linewidth)
#             prev = corner

#     # Draw the sides
#     for i in range(4):
#         cv2.line(im,
#                  (int(corners.T[i][0]), int(corners.T[i][1])),
#                  (int(corners.T[i + 4][0]), int(corners.T[i + 4][1])),
#                  colors[2][::-1], linewidth)

#     # Draw front (first 4 corners) and rear (last 4 corners) rectangles(3d)/lines(2d)
#     draw_rect(corners.T[:4], colors[0][::-1])
#     draw_rect(corners.T[4:], colors[1][::-1])

#     # Draw line indicating the front
#     center_bottom_forward = np.mean(corners.T[2:4], axis=0)
#     center_bottom = np.mean(corners.T[[2, 3, 7, 6]], axis=0)
#     cv2.line(im,
#              (int(center_bottom[0]), int(center_bottom[1])),
#              (int(center_bottom_forward[0]), int(center_bottom_forward[1])),
#              colors[0][::-1], linewidth)

#     # Add some text to the bounding box
#     # Create text to be written
#     attribute_tokens =  nusc.get('sample_annotation', box.token)['attribute_tokens']
#     text = box.name
#     for attribute_token in attribute_tokens:
#         attribute = nusc.get('attribute', attribute_token)
#         text += '\n' + attribute['name']
#     visibility = nusc.get('sample_annotation', box.token)['visibility_token']
#     text += '\nvisibility ' + visibility
#     # Setup multi line text
#     font = cv2.FONT_HERSHEY_COMPLEX
#     font_scale = 0.5
#     thickness = 1
#     text_size, _ = cv2.getTextSize(box.name, font, font_scale, thickness)
#     line_height = text_size[1] + 5
#     center = np.mean(corners.T[:], axis=0)
#     y0 = int(center[1])
#     for i, text_line in enumerate(text.split('\n')):
#         y = y0 + i * line_height
#         cv2.putText(im, text_line, (int(center[0]), y),
#                     cv2.FONT_HERSHEY_COMPLEX, font_scale, colors[0][::-1], thickness, cv2.LINE_AA)
    
    


In [5]:
# def render_birth_and_prebirth(prev_im: np.ndarray,
#                               current_im: np.ndarray,
#                               resize_factor: float) -> np.ndarray:
    
#     prev_im_shrink = cv2.resize(prev_im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
#     current_im_shrink = cv2.resize(current_im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
#     return np.vstack([prev_im_shrink, current_im_shrink])


In [6]:
    # break_all = False
    # break_scene = False
    # unique_instances = []
    # minimum_visibility = 2
    # while has_more_frames:


    #     # Get annotations, boxes, camera_intrinsic
    #     # When using BoxVisibility.ALL objects are detected too late in certain cases
    #     impath, boxes, camera_intrinsic = nusc.get_sample_data(current_sd_rec['token'], box_vis_level=BoxVisibility.ANY)
    #     if not osp.exists(impath):
    #         raise Exception('Error: Missing image %s' % impath)

    #     current_im = cv2.imread(impath)
    #     if current_sd_rec['is_key_frame']:
    #         cv2_put_multi_line_text(current_im, "KEYFRAME", (int(current_im.shape[1]/2), int(current_im.shape[0]/2-80)), (255, 0, 0))
    #     im_contains_unique_instance = False
    #     is_first_frame = False
    #     if prev_im is None:
    #         prev_im = current_im.copy()
    #         is_first_frame = True
    #         cv2_put_multi_line_text(prev_im, "First frame", (int(prev_im.shape[1]/2), int(prev_im.shape[0]/2-25)), (0, 0, 0))
    #         cv2_put_multi_line_text(current_im, "First frame", (int(prev_im.shape[1]/2), int(prev_im.shape[0]/2-25)), (0, 0, 0))

    #     if current_sd_rec['is_key_frame']:    
    #         skip_box = False
    #         for box in boxes:
    #             sample_annotation = nusc.get('sample_annotation', box.token)
    #             instance = nusc.get('instance', sample_annotation['instance_token'])
    #             category = nusc.get('category', instance['category_token'])
    #             # Exclude certain objects
    #             if category['index'] > 8 and category['index'] < 14:
    #                 continue
    #             for attribute_token in sample_annotation['attribute_tokens']:
    #                 attribute_name = nusc.get('attribute', attribute_token)['name']
    #                 if 'without_rider' in attribute_name or 'sitting' in attribute_name:
    #                     skip_box = True
    #             if skip_box:
    #                 continue

    #             # Gather unique instances
    #             instance_token = sample_annotation['instance_token']

    #             # print(current_sd_rec)
    #             if instance_token not in unique_instances:
    #                 im_contains_unique_instance = True
    #                 unique_instances.append(instance_token)
    #                 # Check if this is the first frame of the scene, which means that the object could not have been born
    #                 if current_sd_rec['timestamp'] == first_sd_rec['timestamp']:
    #                     pass
    #                 else:
    #                     # This box can mean a potential object birth
    #                     c = nusc.explorer.get_color(box.name)
    #                     box_render_cv2_text(box, nusc, current_im, view=camera_intrinsic, normalize=True, colors=(c, c, c))

    #                     font = cv2.FONT_HERSHEY_COMPLEX
    #                     font_scale = 0.8
    #                     thickness = 1
    #                     text_line = "visibility " + str(sample_annotation['visibility_token'])
    #                     cv2.putText(current_im, text_line, (int(current_im.shape[1]/2), int((current_im.shape[0]/2))),
    #                                 cv2.FONT_HERSHEY_COMPLEX, font_scale, c[::-1], thickness, cv2.LINE_AA)
            
    #     birth_im = render_birth_and_prebirth(prev_im, current_im, 0.7)
    #     # if int(sample_annotation['visibility_token']) < minimum_visibility:
    #     #     continue            
    #     cv2.imshow("birth and prebirth image", birth_im)
    #     if im_contains_unique_instance or is_first_frame:
    #         key = cv2.waitKey(0)
    #     else:
    #         key = cv2.waitKey(1)
    #     if key == 27:  # if ESC is pressed, exit.
    #         cv2.destroyAllWindows()
    #         break_all = True
    #         break_scene = True
    #         break

    #                 # for attribute_token in sample_annotation['attribute_tokens']:
    #                 #     attribute = nusc.get('attribute', attribute_token)
    #                     # print("attribute", attribute)

    #     # cv2.imshow(window_name, current_im)
    #     # key = cv2.waitKey(milliseconds)
    #     # if key == 32: # if space is pressed, pause.
    #     #     key = cv2.waitKey()
    #     # if key == 27: # if ESC is pressed, exit.
    #     #     break_all = True
    #     #     cv2.destroyAllWindows()
    #     #     break
    #     # if key == 110: # if n is pressed, skip current scene.
    #     #     cv2.destroyAllWindows()
    #     #     break

    #     if break_scene:
    #         break

    #     if current_sd_rec['next'] == '':
    #         has_more_frames = False
    #         cv2.destroyAllWindows()
    #     else:
    #         prev_im = current_im    
    #         prev_sd_rec = current_sd_rec
    #         current_sd_rec = nusc.get('sample_data', current_sd_rec['next'])
        
    # if break_all:
    #     break

In [7]:
def compute_MAE(saliency_map, groundtruth):
    assert(saliency_map.shape == groundtruth.shape)
    return 1/(saliency_map.size) * np.sum(np.abs(saliency_map - groundtruth))


def overlay_image_and_saliency_as_heatmap(im: np.ndarray,
                                          saliency: np.ndarray,
                                          resize_factor: float) -> np.ndarray:

    assert(im.shape == saliency.shape)
    
    im_resize = cv2.resize(im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    saliency_resize = cv2.resize(saliency, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    if im_resize.dtype != np.uint8:
        im_resize = np.asarray(im_resize, dtype=np.uint8)
    if saliency_resize.dtype != np.uint8:
        saliency_resize = np.asarray(saliency_resize * 255, dtype=np.uint8)
    if len(im_resize.shape) != len(saliency_resize.shape):
        if len(saliency_resize.shape) == 2:  # grayscale
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_GRAY2BGR)  # convert to color
        elif len(saliency_resize.shape) == 3:  # color
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_BGR2GRAY)  # convert to grayscale


    saliency_resize = cv2.applyColorMap(saliency_resize, cv2.COLORMAP_JET)
    print("im", im_resize.shape)
    print("saliency ", saliency_resize.shape)
    alpha = 0.7
    beta = 0.3
    gamma = 0
    return cv2.addWeighted(im_resize, alpha, saliency_resize, beta, gamma)

def overlay_image_and_saliency_as_heatmap_and_groundtruth(im: np.ndarray,
                                          saliency: np.ndarray,
                                          groundtruth: np.ndarray,
                                          resize_factor: float) -> np.ndarray:
    print(im.shape[0:2], saliency.shape[0:2], groundtruth.shape[0:2])
    assert(im.shape[0:2] == saliency.shape[0:2] and im.shape[0:2] ==
           groundtruth.shape[0:2] and saliency.shape[0:2] == groundtruth.shape[0:2])
    im_resize = cv2.resize(im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    saliency_resize = cv2.resize(saliency, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    groundtruth_resize = cv2.resize(groundtruth, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)


    if im_resize.dtype != np.uint8:
        im_resize = np.asarray(im_resize, dtype=np.uint8)
    if saliency_resize.dtype != np.uint8:
        saliency_resize = np.asarray(saliency_resize * 255, dtype=np.uint8)
    groundtruth_resize = np.asarray(groundtruth_resize * 255, dtype=np.uint8)
    
    # conver to color
    if len(im_resize.shape) != len(saliency_resize.shape):
        if len(saliency_resize.shape) == 2:  # grayscale
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_GRAY2BGR)  # convert to color
        elif len(saliency_resize.shape) == 3:  # color
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_BGR2GRAY)  # convert to grayscale
    saliency_resize = cv2.applyColorMap(saliency_resize, cv2.COLORMAP_JET)
    
    groundtruth_resize = cv2.cvtColor(groundtruth_resize, cv2.COLOR_GRAY2BGR)
    
    alpha = 0.7
    beta = 0.3
    gamma = 0
    res = cv2.addWeighted(im_resize, alpha, saliency_resize, beta, gamma)
    res = cv2.addWeighted(res, alpha, groundtruth_resize, beta, gamma)
    return res

def display_im(im: np.ndarray,
               window_name: str):
    cv2.imshow(window_name, im)
    key = cv2.waitKey(0)
    global break_all
    if key == 27:
        break_all = True
    cv2.destroyWindow(window_name)


In [8]:
import matplotlib.pyplot as plt

import os
import sys
import pandas as pd
import matplotlib.cm as colormap
from PIL import Image  # to read png ground truth images in the correct uint8 format (mpimg loads as float)
import cv2

wanted_channel = 'CAM_FRONT'

MAE_df = pd.DataFrame()

break_all = False

# initialize OpenCV's static saliency spectral residual detector and
sr = cv2.saliency.StaticSaliencySpectralResidual_create()

sr_h32_w_106 = cv2.saliency.StaticSaliencySpectralResidual_create()
sr_h32_w_106.setImageHeight(32)
sr_h32_w_106.setImageWidth(106)

sr_h64_w_212 = cv2.saliency.StaticSaliencySpectralResidual_create()
sr_h64_w_212.setImageHeight(64)
sr_h64_w_212.setImageWidth(212)

sr_h128_w_424 = cv2.saliency.StaticSaliencySpectralResidual_create()
sr_h128_w_424.setImageHeight(128)
sr_h128_w_424.setImageWidth(424)

front_counter = 0
for idx, sample in enumerate(nuim.sample):
    
    sd_tokens = nuim.get_sample_content(sample['token'])
    
    for sd_token in sd_tokens:
        # only check key frames
        sd = nuim.get('sample_data', sd_token)
        if not sd['is_key_frame']:
            continue
        # check channel
        calibrated_sensor = nuim.get('calibrated_sensor', sd['calibrated_sensor_token'])
        # print(nuim.get('calibrated_sensor', sd['calibrated_sensor_token']))
        sensor = nuim.get('sensor', calibrated_sensor['sensor_token'])
        channel = sensor['channel']
        if channel != wanted_channel:
            continue
        
        front_counter += 1
        print("Evaluating front_counter #{:06d} sample #{:06d}".format(front_counter, idx))
        semantic_mask, instance_mask = nuim.get_segmentation(sd_token)

        filename = os.path.join(nuim.dataroot, sd['filename'])
        im = cv2.imread(filename, cv2.IMREAD_COLOR)

        frame_mae_df = pd.DataFrame({'file': filename, 'ooi_cat': 'traffic_participants'}, index=[0])
        gt_mask = np.zeros(semantic_mask.shape)
        gt_indxs = np.where( ((semantic_mask > 0) & (semantic_mask < 9)) | ((semantic_mask > 13) & (semantic_mask < 24)) )
        gt_mask[gt_indxs] = 1
        gt_mask_num_pixels = len(gt_indxs[0])
        frame_mae_df['mask_size_px'] = gt_mask_num_pixels

        # calculate mean absolute errors
        (success, map_h32_w106) = sr_h32_w_106.computeSaliency(im)
        (success, map_h64_w212) = sr_h64_w_212.computeSaliency(im)
        (success, map_h128_w424) = sr_h128_w_424.computeSaliency(im)

        MAE_sr32 = compute_MAE(map_h32_w106, gt_mask)
        MAE_sr64 = compute_MAE(map_h64_w212, gt_mask)
        MAE_sr128 = compute_MAE(map_h128_w424, gt_mask)
        frame_mae_df['SR32'] = MAE_sr32
        frame_mae_df['SR64'] = MAE_sr64
        frame_mae_df['SR128'] = MAE_sr128


        # cv2.imshow("image_raw", im)
        display_im_h32_w106 = overlay_image_and_saliency_as_heatmap_and_groundtruth(im, map_h32_w106, gt_mask, 1.1)
        display_im_h64_w212 = overlay_image_and_saliency_as_heatmap_and_groundtruth(im, map_h64_w212, gt_mask, 1.1)
        display_im_h128_w424 = overlay_image_and_saliency_as_heatmap_and_groundtruth(im, map_h128_w424, gt_mask, 1.1)
        cv2_put_multi_line_text(display_im_h32_w106, "mae sr32 " + str(MAE_sr32), (300, 300), (0,255,0))
        cv2_put_multi_line_text(display_im_h64_w212, "mae sr64 " + str(MAE_sr64), (300, 300), (0,255,0))
        cv2_put_multi_line_text(display_im_h128_w424, "mae sr128 " + str(MAE_sr128), (300, 300),  (0,255,0))

        display_im(display_im_h32_w106, "display_im_h32_w106")
        display_im(display_im_h64_w212, "display_im_h64_w212")
        display_im(display_im_h128_w424, "display_im_h128_w424")

        for H in [8, 16, 32]:
            sr.setImageHeight(H)

            W = int(np.round(H*(1600/900), 0))
            sr.setImageWidth(W)

            (success, map_low_res) = sr.computeSaliency(im)

            # context saliency map
            csm = (1-map_low_res)

            # thresholded context saliency map
            csm_th = csm.copy()
            thres = 0.7
            csm_th[(csm < thres)] = 0

            comb_map_32 = csm_th*map_h32_w106
            comb_map_32 = comb_map_32/comb_map_32.max()

            comb_map_64 = csm_th*map_h64_w212
            comb_map_64 = comb_map_64/comb_map_64.max()

            comb_map_128 = csm_th*map_h128_w424
            comb_map_128 = comb_map_128/comb_map_128.max()

            # compute MAE
            MAE_csm = compute_MAE(csm_th, gt_mask)
            MAE_comb32 = compute_MAE(comb_map_32, gt_mask)
            MAE_comb64 = compute_MAE(comb_map_64, gt_mask)
            MAE_comb128 = compute_MAE(comb_map_128, gt_mask)

            # frame_mae_df['C'+str(H)] = MAE_csm
            # frame_mae_df['C'+str(H)+'xSR32'] = MAE_comb32
            # frame_mae_df['C'+str(H)+'xSR64'] = MAE_comb64
            # frame_mae_df['C'+str(H)+'xSR128'] = MAE_comb128

            # display_im_MAE_csm = overlay_image_and_saliency_as_heatmap()
            # display_im_MAE_comb32 = overlay_image_and_saliency_as_heatmap()
            # display_im_MAE_comb64 = overlay_image_and_saliency_as_heatmap()
            # display_im_MAE_comb128 = overlay_image_and_saliency_as_heatmap()

        # MAE_df = pd.concat([MAE_df, frame_mae_df], axis=0,  ignore_index=True)
        
        if break_all:
            break
    
    if break_all:
        break
    #     if idx > 100:
    #         break_all = True
    # if break_all:
    #     break
    # has_more_frames = True

# MAE_df['mask_size'] = pd.cut(MAE_df.mask_size_px.values, 3, labels = ['small', 'medium', 'large'])

# base_results_path = '/data/datasets/nuimages/results/contextual_prior'
# hdf_filename = os.path.join(base_results_path, 'MAE_per_category.h5')
# hdf_p = pd.HDFStore(hdf_filename)
# hdf_p.put('MAE_per_category', MAE_df, format='table')
# hdf_p.close()  # closes the file
# print('saved to HDF')

# csv_filename = os.path.join(base_results_path, 'MAE_per_category.csv')
# MAE_df.to_csv(csv_filename, index=False)
# print('saved to CSV')


Evaluating front_counter #000001 sample #000012
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
Evaluating front_counter #000002 sample #000016
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
Evaluating front_counter #000003 sample #000022
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
Evaluating front_counter #000004 sample #000029
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
Evaluating front_counter #000005 sample #000031
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
Evaluating front_counter #000006 sample #000046
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
(900, 1600) (900, 1600) (900, 1600)
Evaluating front_counter #000007 sample #000047
(900, 1600) (900